<a href="https://colab.research.google.com/github/DragonZ2013/TokenizerAnonimizer/blob/main/TokenizerAnonymizerUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()
#Add the created anonymizer scripts to the colab project (currently python and js)

Saving anonymize_js.js to anonymize_js.js
Saving anonymize_python.py to anonymize_python.py


In [ ]:
!npm install acorn estraverse escodegen

⠙⠹⠸⠼⠴⠦
up to date, audited 7 packages in 865ms
⠦
found 0 vulnerabilities
⠦

In [ ]:
!chmod +x anonymize_python.py

chmod: cannot access 'anonymize_python.py': No such file or directory


In [ ]:
import ipywidgets as widgets
from IPython.display import display, FileLink, Javascript
import tempfile
import subprocess
from pathlib import Path

# Language dropdown (you can extend this list)
language_dropdown = widgets.Dropdown(
    options=['python', 'javascript'],
    value='python',
    description='Language:'
)

# Upload widget
upload = widgets.FileUpload(multiple=False)


# Textarea for anonymized output
anonymized_output = widgets.Textarea(
    value='',
    placeholder='Anonymized code will appear here...',
    description='Output:',
    layout=widgets.Layout(width='100%', height='300px')
)

# Buttons
analyze_button = widgets.Button(description='Anonymize', button_style='success')
copy_button = widgets.Button(description='Copy Output')
download_button = widgets.Button(description='Download Output')

output_area = widgets.Output()

# --- Function to update accepted file types based on selected language ---
def update_upload_accept(change):
    # Clear the uploaded file when the language is changed
    upload.value.clear()

    if language_dropdown.value == 'python':
        upload.accept = '.py,.txt'
    elif language_dropdown.value == 'javascript':
        upload.accept = '.js,.txt'
    else:
        upload.accept = '.txt'

    # Clear any existing output and reset textarea
    anonymized_output.value = ''
    output_area.clear_output()

# Link the update to the dropdown value change
language_dropdown.observe(update_upload_accept, names='value')
update_upload_accept(None)  # Call this initially to set the default

# --- Main Anonymization Logic ---
def run_external_anonymizer(language, input_path, output_path):
    try:
        if language == "python":
            cmd = f"python3 anonymize_python.py '{input_path}' '{output_path}'"
        elif language == "javascript":
            cmd = f"node anonymize_js.js '{input_path}' > '{output_path}'"
        else:
            raise ValueError("Unsupported language")

        subprocess.run(cmd, shell=True, check=True)

    except subprocess.CalledProcessError as e:
        raise RuntimeError(f"Anonymizer failed: {e}")

# --- Event Handlers ---
def on_analyze_clicked(b):
    analyze_button.disabled = True
    analyze_button.description = "Processing..."
    output_area.clear_output()

    if not upload.value:
        anonymized_output.value = "Please upload a file first."
        analyze_button.disabled = False
        analyze_button.description = "Anonymize"
        return

    # Save uploaded file
    for name, file_info in upload.value.items():
        input_path = Path(tempfile.gettempdir()) / name
        with open(input_path, "wb") as f:
            f.write(file_info["content"])

        output_path = Path(tempfile.gettempdir()) / f"{name}_anon.txt"

        try:
            run_external_anonymizer(language_dropdown.value, input_path, output_path)
            anonymized_output.value = Path(output_path).read_text()
        except Exception as e:
            anonymized_output.value = f"Anonymizer failed:\n{str(e)}"

    analyze_button.disabled = False
    analyze_button.description = "Anonymize"


def on_copy_clicked(b):
    display(Javascript("""
    navigator.clipboard.writeText(document.querySelector('textarea').value);
    alert("Copied to clipboard!");
    """))

def on_download_clicked(b):
    from IPython.display import Javascript
    import base64

    content = anonymized_output.value
    if not content.strip():
        with output_area:
            output_area.clear_output()
            print("No content to download.")
        return

    b64 = base64.b64encode(content.encode()).decode()
    filename = "anonymized_output.txt"
    payload = f"data:text/plain;base64,{b64}"

    display(Javascript(f"""
        var a = document.createElement('a');
        a.href = "{payload}";
        a.download = "{filename}";
        document.body.appendChild(a);
        a.click();
        document.body.removeChild(a);
    """))

# Attach event listeners
analyze_button.on_click(on_analyze_clicked)
copy_button.on_click(on_copy_clicked)
download_button.on_click(on_download_clicked)

# --- UI Layout ---
ui = widgets.VBox([
    widgets.Label("Language-Aware Code Anonymizer"),
    language_dropdown,
    upload,
    analyze_button,
    anonymized_output,
    widgets.HBox([copy_button, download_button]),
    output_area
])

display(ui)


<IPython.core.display.Javascript object>